Import Libraries

In [ ]:
from google.colab import drive
import json
import numpy as np
import os
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, cohen_kappa_score, confusion_matrix

In [ ]:
drive.mount('/content/drive')

# cross validation's final results of

In [ ]:
# Folder where the results are saved
base_save_path = 'Insert path to saved results'
num_splits = 10

# Dictionary to accumulate metrics
metrics = {
    "accuracy": [],
    "f1": [],
    "precision": [],
    "recall": [],
    "cohen_kappa": []
}

# Loop over all splits
for i in range(1, num_splits + 1):
    results_file = os.path.join(base_save_path, f'Cross {i}', 'test_results.json')

    if os.path.exists(results_file):
        with open(results_file, 'r') as f:
            res = json.load(f)

        # Append main metrics
        for k in metrics.keys():
            if k in res:
                metrics[k].append(res[k])
    else:
        print(f" Missing results for Cross {i}")

# Compute mean and standard deviation
final_results = {}
for k, values in metrics.items():
    if len(values) > 0:
        final_results[k] = {
            "mean": np.mean(values),
            "std": np.std(values)
        }

# Print summary
print("\n=== FINAL CROSS-VALIDATION RESULTS ===")
for k, v in final_results.items():
    print(f"{k}: {v['mean']:.4f} ± {v['std']:.4f}")

# Save summary as JSON
summary_file = os.path.join(base_save_path, 'cross_validation_summary.json')
with open(summary_file, 'w') as f:
    json.dump(final_results, f, indent=4)

print(f"\nSummary saved at: {summary_file}")


# Results without oultiers

In [ ]:
# Folder where the results are saved
base_save_path = 'Insert path to saved results'
num_splits = 10

# Dictionary to accumulate metrics
metrics = {
    "accuracy": [],
    "f1": [],
    "precision": [],
    "recall": [],
    "cohen_kappa": []
}

# Loop over all splits
for i in range(1, num_splits + 1):
    results_file = os.path.join(base_save_path, f'Cross {i}', 'test_results.json')

    if os.path.exists(results_file):
        with open(results_file, 'r') as f:
            res = json.load(f)

        # Append main metrics
        for k in metrics.keys():
            if k in res:
                metrics[k].append(res[k])
    else:
        print(f" Missing results for Cross {i}")

# Identify and remove folds with best and worst F1 scores
f1_scores = np.array(metrics["f1"])
best_idx = np.argmax(f1_scores)
worst_idx = np.argmin(f1_scores)

print(f"\nRemoving outliers:")
print(f" - Best fold (F1={f1_scores[best_idx]:.4f}) → removed")
print(f" - Worst fold (F1={f1_scores[worst_idx]:.4f}) → removed")

# Remove those indices from all metrics
indices_to_keep = [i for i in range(len(f1_scores)) if i not in [best_idx, worst_idx]]
metrics_filtered = {k: [v[i] for i in indices_to_keep] for k, v in metrics.items()}

# Compute mean and standard deviation after removal
final_results_filtered = {}
for k, values in metrics_filtered.items():
    if len(values) > 0:
        final_results_filtered[k] = {
            "mean": np.mean(values),
            "std": np.std(values)
        }

# Print final summary
print("\n=== FINAL CROSS-VALIDATION RESULTS (without outliers) ===")
for k, v in final_results_filtered.items():
    print(f"{k}: {v['mean']:.4f} ± {v['std']:.4f}")

# Save summary as JSON
summary_file_filtered = os.path.join(base_save_path, 'cross_validation_summary_no_outliers.json')
with open(summary_file_filtered, 'w') as f:
    json.dump(final_results_filtered, f, indent=4)

print(f"\nSummary (without outliers) saved at: {summary_file_filtered}")



# Global analysis

In [ ]:
# Aggregated evaluation over all predictions
all_true = []
all_pred = []

for i in range(1, num_splits + 1):
    folder = os.path.join(base_save_path, f'Cross {i}')
    true_path = os.path.join(folder, 'true_labels.npy')
    pred_path = os.path.join(folder, 'predicted_labels.npy')

    if os.path.exists(true_path) and os.path.exists(pred_path):
        true_labels = np.load(true_path)
        pred_labels = np.load(pred_path)
        all_true.extend(true_labels)
        all_pred.extend(pred_labels)
    else:
        print(f" Missing predictions for Cross {i}")

if len(all_true) > 0:
    all_true = np.array(all_true)
    all_pred = np.array(all_pred)

    # Compute global metrics
    acc_global = accuracy_score(all_true, all_pred)
    f1_global = f1_score(all_true, all_pred, average="weighted")
    prec_global = precision_score(all_true, all_pred, average="weighted")
    rec_global = recall_score(all_true, all_pred, average="weighted")
    kappa_global = cohen_kappa_score(all_true, all_pred)
    cm_global = confusion_matrix(all_true, all_pred)

    print("\n=== GLOBAL EVALUATION ACROSS ALL SPLITS ===")
    print(f"Accuracy: {acc_global:.4f}")
    print(f"F1-score: {f1_global:.4f}")
    print(f"Precision: {prec_global:.4f}")
    print(f"Recall: {rec_global:.4f}")
    print(f"Cohen’s Kappa: {kappa_global:.4f}")
    print("Confusion Matrix:\n", cm_global)

    # Save global results as JSON
    global_results = {
        "accuracy": acc_global,
        "f1": f1_global,
        "precision": prec_global,
        "recall": rec_global,
        "cohen_kappa": kappa_global,
        "confusion_matrix": cm_global.tolist()
    }

    global_file = os.path.join(base_save_path, 'cross_validation_global_results.json')
    with open(global_file, 'w') as f:
        json.dump(global_results, f, indent=4)

    print(f"\nGlobal results saved at: {global_file}")

